# Module 2: Build a RAG with Cortex Search

 
To run, first install the following packages: `snowflake-ml-python`, `snowflake.core`, `trulens-core`, `trulens-providers-cortex`, `trulens-connectors-snowflake`

## Create the database, tables and warehouse

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
CREATE DATABASE IF NOT EXISTS cortex_search_tutorial_db;

CREATE OR REPLACE WAREHOUSE cortex_search_tutorial_wh WITH
     WAREHOUSE_SIZE='X-SMALL'
     AUTO_SUSPEND = 120
     AUTO_RESUME = TRUE
     INITIALLY_SUSPENDED=TRUE;

 USE WAREHOUSE cortex_search_tutorial_wh;

Note:

The CREATE DATABASE statement creates a database. The database automatically includes a schema named PUBLIC.

The CREATE WAREHOUSE statement creates an initially suspended warehouse.

## Get PDF data

You will use a sample dataset of the Federal Open Market Committee (FOMC) meeting minutes for this example. This is a sample of twelve 10-page documents with meeting notes from FOMC meetings from 2023 and 2024. Download the files directly from your browser by following this link:

[FOMC minutes sample](https://drive.google.com/file/d/1C6TdVjy6d-GnasGO6ZrIEVJQRcedDQxG/view)

The complete set of FOMC minutes can be found at the [US Federal Reserve’s website](https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm).

Note: In a non-classroom setting, you would bring your own data, possibly already in a Snowflake stage.

## Load data into Snowflake stage

In [ ]:
CREATE STAGE IF NOT EXISTS cortex_search_tutorial_db.public.fomc
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

Now upload the dataset. You can upload the dataset in Snowsight or using SQL. To upload in Snowsight:

1. Sign in to Snowsight.

2. Select Data in the left-side navigation menu.

3. Select your database cortex_search_tutorial_db.

4. Select your schema public.

5. Select Stages and select fomc.

6. On the top right, Select the + Files button.

7. Drag and drop files into the UI or select Browse to choose a file from the dialog window.

8. Select Upload to upload your file.

## Verify the PDF Files are uploaded to stage

In [ ]:
ls @cortex_search_tutorial_db.public.fomc

## Parse PDF Files

In [ ]:
CREATE OR REPLACE TABLE CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.PARSED_FOMC_CONTENT AS SELECT 
      relative_path,
      TO_VARCHAR(
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
          @cortex_search_tutorial_db.public.fomc, 
          relative_path, 
          {'mode': 'LAYOUT'}
        ) :content
      ) AS parsed_text
    FROM directory(@cortex_search_tutorial_db.public.fomc)
    WHERE relative_path LIKE '%.pdf'

In [ ]:
SELECT * FROM CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.PARSED_FOMC_CONTENT LIMIT 2

## Chunk text

In [ ]:
CREATE OR REPLACE TABLE CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.CHUNKED_FOMC_CONTENT (
    file_name VARCHAR,
    CHUNK VARCHAR
);

INSERT INTO CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.CHUNKED_FOMC_CONTENT (file_name, CHUNK)
SELECT
    relative_path,
    c.value AS CHUNK
FROM
    CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.PARSED_FOMC_CONTENT,
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
        parsed_text,
        'markdown',
        1800,
        250
    )) c;

In [ ]:
SELECT * FROM CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.CHUNKED_FOMC_CONTENT LIMIT 10

## Create Search Service

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.FOMC_SEARCH_SERVICE
    ON chunk
    WAREHOUSE = cortex_search_tutorial_wh
    TARGET_LAG = '1 minute'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
    SELECT
        file_name,
        chunk
    FROM CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.CHUNKED_FOMC_CONTENT
    );

## Use the Search Service

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
import os
from snowflake.core import Root
from typing import List
from snowflake.snowpark.session import Session

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)

        search_service = (root
          .databases["CORTEX_SEARCH_TUTORIAL_DB"]
          .schemas["PUBLIC"]
          .cortex_search_services["FOMC_SEARCH_SERVICE"]
        )
        resp = search_service.search(
          query=query,
          columns=["chunk"],
          limit=self._limit_to_retrieve
        )

        if resp.results:
            return [curr["chunk"] for curr in resp.results]
        else:
            return []

In [ ]:
retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=3)

retrieved_context = retriever.retrieve(query="how was inflation expected to evolve in 2024?")

retrieved_context

## Turn on OpenTelemetry Tracing

In [ ]:
import os
os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
create or replace database observability_db;
use database observability_db;
create or replace schema observability_schema;
use schema observability_schema;

## Create the RAG with instrumentation

In [ ]:
from snowflake.cortex import complete
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes

class RAG:

    def __init__(self):
        self.retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=4)

    @instrument(
        span_type=SpanAttributes.SpanType.RETRIEVAL,
        attributes={
            SpanAttributes.RETRIEVAL.QUERY_TEXT: "query",
            SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: "return",
            }
    )
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)


    @instrument(
        span_type=SpanAttributes.SpanType.GENERATION)
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
          You are an expert assistant extracting information from context provided.
          Answer the question in long-form, fully and completely, based on the context. Do not hallucinate.
          If you don´t have the information just say so.
          Context: {context_str}
          Question:
          {query}
          Answer:
        """
        response = ""
        stream = complete("mistral-large2", prompt, stream = True)
        for update in stream:    
          response += update
          print(update, end = '')
        return response

    @instrument(
        span_type=SpanAttributes.SpanType.RECORD_ROOT, 
        attributes={
            SpanAttributes.RECORD_ROOT.INPUT: "query",
            SpanAttributes.RECORD_ROOT.OUTPUT: "return",
        })
    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG()

## Query the RAG

In [ ]:
response = rag.query("how was inflation expected to evolve in 2024?")

## Register the App

In [ ]:
from trulens.apps.app import TruApp
from trulens.connectors.snowflake import SnowflakeConnector

tru_snowflake_connector = SnowflakeConnector(snowpark_session=session)

app_name = "fed_reserve_rag"
app_version = "cortex_search"

tru_rag = TruApp(
        rag,
        app_name=app_name,
        app_version=app_version,
        connector=tru_snowflake_connector
    )

## Load (or create) test set

In [ ]:
import pandas as pd

experiment1_data = {
    'query': [
        "What were the key points discussed in the FOMC meeting in January 2023?",
        "How did the FOMC view the economic outlook in mid-2023?",
        "What were the inflation expectations for the end of 2023?",
        "What were the main topics in the FOMC meeting in February 2024?",
        "How did the FOMC assess the labor market in mid-2024?",
        "What were the GDP growth projections for the end of 2024?",
        "What was the fed funds rate target range effective september 19, 2024",
        "What were the inflation expectations for 2025?"
    ]
}

experiment1_df = pd.DataFrame(experiment1_data)
print(experiment1_df)

## Configure and add experment run

In [ ]:
from trulens.core.run import Run
from trulens.core.run import RunConfig

run_name = "experiment_1_run"

run_config = RunConfig(
    run_name=run_name,
    dataset_name="experiment1_data",
    description="Questions about the Federal Open Market Committee meetings",
    label="fomc_rag_eval",
    source_type="DATAFRAME",
    dataset_spec={
        "input": "query",
    },
)

run: Run = tru_rag.add_run(run_config=run_config)

## Describe the run

In [ ]:
run.describe()

## Start the run

In [ ]:
run.start(input_df=experiment1_df)

## Compute metrics on the run

In [ ]:
run.compute_metrics([
    "coherence",
    "answer_relevance",
    "context_relevance",
    "groundedness",
])

## Navigate to AI > Applications to view AI Observability

You may notice that there is low context relevance and groundedness scores for queries about 2025.

This is a common symptom of an out-of-date knowledge base, where the user has asked a question of knowledge that our search service doesn't yet know about.

To address this issue, we need to add new knowledge to our RAG via the Cortex Search service.

## Automatic Processing of New Documents

We can use Snowflake features Streams and Task to automatically process new PDF files as they are added into Snowflake.

The tasks and streams will utilize the same parsing and chunking queries we built earlier in the notebook.

In [ ]:
CREATE OR REPLACE STREAM cortex_search_tutorial_db.public.fomc_docs_stream
ON STAGE cortex_search_tutorial_db.public.fomc;

In [ ]:
CREATE OR REPLACE TASK cortex_search_tutorial_db.public.fomc_parse_task
    WAREHOUSE = cortex_search_tutorial_wh
    SCHEDULE = '1 MINUTE'
    WHEN SYSTEM$STREAM_HAS_DATA('cortex_search_tutorial_db.public.fomc_docs_stream')
AS
INSERT INTO CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.PARSED_FOMC_CONTENT(RELATIVE_PATH, PARSED_TEXT)
SELECT
      relative_path,
      TO_VARCHAR(
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
          @cortex_search_tutorial_db.public.fomc, 
          relative_path, 
          {'mode': 'LAYOUT'}
        ) :content
      ) AS parsed_text
    FROM directory(@cortex_search_tutorial_db.public.fomc)
    WHERE relative_path LIKE '%.pdf'

In [ ]:
CREATE OR REPLACE STREAM cortex_search_tutorial_db.public.parsed_fomc_stream
ON TABLE cortex_search_tutorial_db.public.parsed_fomc_content;

In [ ]:
CREATE OR REPLACE TASK cortex_search_tutorial_db.public.fomc_chunk_task
    WAREHOUSE = cortex_search_tutorial_wh
    SCHEDULE = '1 MINUTE'
    WHEN SYSTEM$STREAM_HAS_DATA('cortex_search_tutorial_db.public.parsed_fomc_stream')
AS
INSERT INTO CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.CHUNKED_FOMC_CONTENT(FILE_NAME, CHUNK)
SELECT
    s.RELATIVE_PATH AS FILE_NAME,
    chunk.VALUE AS CHUNK
FROM cortex_search_tutorial_db.public.parsed_fomc_stream AS s,
     LATERAL FLATTEN(
       INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
                  s.PARSED_TEXT,
                  'markdown',
                  1800,
                  250
                )
     ) chunk;

In [ ]:
alter task cortex_search_tutorial_db.public.fomc_parse_task resume;
alter task cortex_search_tutorial_db.public.fomc_chunk_task resume;

In [ ]:
response = rag.query("when was the most recent beige book and what was its biggest takeaway?")